In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_cata_url(catagory):
    ''' Takes in a string containing a player card catagory
        Scrapes arkhamdb
        Returns a list of urls for containing information on each card of that catagory'''

    # get url for page to scrape
    url = f'https://arkhamdb.com/find?q=t%3A{catagory}&decks=player'

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page
    results = soup.find(id='list')

    results = results.find_all('a', class_='card-tip')

    # convert urls to string and make a list
    results = [str(result['href']) for result in results]

    return results

In [ ]:
def get_icons(results):
    '''Takes in request results for an arkhamdb page containing player card data
       Returns a string containing the skill icons printed on that cart'''
      
    icons = ''

    # list containing each icon type
    icon_types = ['wild', 'willpower', 'combat', 'agility', 'intellect']

    # itterate through icon types
    for stat in icon_types:

        # get number of that icon on card from request results
        num_icons = len(results.find_all('span', class_=f'icon icon-{stat} color-{stat}'))

        # add that icon name to a string for each time it appears in request results
        for icon in range(num_icons):

            icons += f'{stat} '
            
    return icons[:-1]

In [3]:
invest = get_cata_url('investigator')
assets = get_cata_url('asset')
events = get_cata_url('event')
skills = get_cata_url('skill')

In [4]:
for cata in [invest, assets, events, skills]:
    
    print(cata[0:6])
    print()

['https://arkhamdb.com/card/02005', 'https://arkhamdb.com/card/01003', 'https://arkhamdb.com/card/01503', 'https://arkhamdb.com/card/01004', 'https://arkhamdb.com/card/01504', 'https://arkhamdb.com/card/03004']

['https://arkhamdb.com/card/60505', 'https://arkhamdb.com/card/60522', 'https://arkhamdb.com/card/07025', 'https://arkhamdb.com/card/07305', 'https://arkhamdb.com/card/03020', 'https://arkhamdb.com/card/52001']

['https://arkhamdb.com/card/03304', 'https://arkhamdb.com/card/52002', 'https://arkhamdb.com/card/06156', 'https://arkhamdb.com/card/08021', 'https://arkhamdb.com/card/60114', 'https://arkhamdb.com/card/60123']

['https://arkhamdb.com/card/09002', 'https://arkhamdb.com/card/03272', 'https://arkhamdb.com/card/03233', 'https://arkhamdb.com/card/05038', 'https://arkhamdb.com/card/04309', 'https://arkhamdb.com/card/09049']



In [60]:
# get url for page to scrape
url = 'https://arkhamdb.com/card/05030'

# create request and soup objects
html = requests.get(url)

soup = BeautifulSoup(html.content, 'html.parser')

# locate urls on page
results = soup.find(id='list')

test_icons = get_icons(results)

title = results.find('a', class_='card-name card-tip').text

cata = results.find('span', class_='card-faction').text

traits = results.find('div', class_='card-traits').text

ability = results.find('div', class_='card-text border-neutral').text

agility intellect 


AttributeError: 'NoneType' object has no attribute 'text'

In [63]:
print(title)

Cunning



In [61]:
test_icons

'agility intellect'

In [36]:
ability

<div class="card-text border-neutral">
<p>Carson Sinclair deck only.</p><p>Commit only to a skill test being performed by another investigator.</p><p>If this test succeeds, the performing investigator draws 1 card. If this test fails, you draw 1 card.</p>
</div>